In [2]:
import pandas as pd #Basic packages for creating dataframes and loading dataset
import numpy as np

import matplotlib.pyplot as plt #Package for visualization

import re #importing package for Regular expression operations

from sklearn.model_selection import train_test_split #Package for splitting the data

from sklearn.preprocessing import LabelEncoder #Package for conversion of categorical to Numerical

from keras.preprocessing.text import Tokenizer #Tokenization
from tensorflow.keras.preprocessing.sequence import pad_sequences #Add zeros or crop based on the length
from keras.models import Sequential #Sequential Neural Network
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D #For layers in Neural Network
from keras.utils import to_categorical

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import pandas as pd

# Load the dataset as a Pandas DataFrame
dataset = pd.read_csv("/content/gdrive/MyDrive/Sentiment (3) (2).csv", header=0)

# Select only the necessary columns 'text' and 'sentiment'
mask = dataset.columns.isin(['text', 'sentiment'])
data = dataset.loc[:, mask]

# Keeping only the necessary columns

In [4]:
data.loc[:, 'text'] = data['text'].apply(lambda x: x.lower())
data.loc[:, 'text'] = data['text'].apply(lambda x: re.sub('[^a-zA-Z0-9\s]', '', x))

In [5]:
for idx, row in data.iterrows():
    row[0] = row[0].replace('rt', ' ') #Removing Retweets

In [6]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ') #Maximum words is 2000 to tokenize sentence
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values) #taking values to feature matrix

In [7]:
X = pad_sequences(X) #Padding the feature matrix

embed_dim = 128 #Dimension of the Embedded layer
lstm_out = 196 #Long short-term memory (LSTM) layer neurons

In [8]:
def createmodel():
    model = Sequential() #Sequential Neural Network
    model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1])) #input dimension 2000 Neurons, output dimension 128 Neurons
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)) #Drop out 20%, 196 output Neurons, recurrent dropout 20%
    model.add(Dense(3,activation='softmax')) #3 output neurons[positive, Neutral, Negative], softmax as activation
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy']) #Compiling the model
    return model
# print(model.summary())

In [9]:
labelencoder = LabelEncoder() #Applying label Encoding on the label matrix
integer_encoded = labelencoder.fit_transform(data['sentiment']) #fitting the model
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42) #67% training data, 33% test data split


In [10]:
batch_size = 32 #Batch size 32
model = createmodel() #Function call to Sequential Neural Network
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2) #verbose the higher, the more messages
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size) #evaluating the model
print(score)
print(acc)

291/291 - 60s - loss: 0.8264 - accuracy: 0.6448 - 60s/epoch - 205ms/step
144/144 - 3s - loss: 0.7498 - accuracy: 0.6689 - 3s/epoch - 23ms/step
0.7498379349708557
0.6688510179519653


In [11]:
print(model.metrics_names) #metrics of the model

['loss', 'accuracy']


1. Save the model and use the saved model to predict on new text data (ex, “A lot of good things are happening. We are respected again throughout the world, and that's a great thing.@realDonaldTrump”)

In [12]:
model.save('sentimentAnalysis.h5') #Saving the model


In [13]:
from keras.models import load_model #Importing the package for importing the saved model
model= load_model('sentimentAnalysis.h5') #loading the saved model

In [14]:
print(integer_encoded)
print(data['sentiment'])

[1 2 1 ... 2 0 2]
0         Neutral
1        Positive
2         Neutral
3        Positive
4        Positive
           ...   
13866    Negative
13867    Positive
13868    Positive
13869    Negative
13870    Positive
Name: sentiment, Length: 13871, dtype: object


In [15]:
# Predicting on the text data
sentence = ['A lot of good things are happening. We are respected again throughout the world, and that is a great thing.@realDonaldTrump']
sentence = tokenizer.texts_to_sequences(sentence) # Tokenizing the sentence
sentence = pad_sequences(sentence, maxlen=28, dtype='int32', value=0) # Padding the sentence
sentiment_probs = model.predict(sentence, batch_size=1, verbose=2)[0] # Predicting the sentence text
sentiment = np.argmax(sentiment_probs)

print(sentiment_probs)
if sentiment == 0:
    print("Neutral")
elif sentiment < 0:
    print("Negative")
elif sentiment > 0:
    print("Positive")
else:
    print("Cannot be determined")

1/1 - 0s - 335ms/epoch - 335ms/step
[0.5661933  0.11436861 0.319438  ]
Neutral


2.Apply GridSearchCV on the source code provided in the class

In [1]:
pip install tensorflow==2.12.0 keras==2.12.0 scikeras==0.11.0

In [16]:
from keras.wrappers.scikit_learn import KerasClassifier #importing Keras classifier
from sklearn.model_selection import GridSearchCV #importing Grid search CV

model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters
batch_size= [10, 20, 40] #hyper parameter batch_size
epochs = [1, 2] #hyper parameter no. of epochs
param_grid= {'batch_size':batch_size, 'epochs':epochs} #creating dictionary for batch size, no. of epochs
grid  = GridSearchCV(estimator=model, param_grid=param_grid) #Applying dictionary with hyper parameters
grid_result= grid.fit(X_train,Y_train) #Fitting the model
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) #best score, best hyper parameters


<ipython-input-16-139d1994cb8f>:4: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=createmodel,verbose=2) #initiating model to test performance by applying multiple hyper parameters


744/744 - 117s - loss: 0.8191 - accuracy: 0.6466 - 117s/epoch - 158ms/step
186/186 - 4s - loss: 0.7520 - accuracy: 0.6740 - 4s/epoch - 20ms/step
744/744 - 117s - loss: 0.8144 - accuracy: 0.6515 - 117s/epoch - 157ms/step
186/186 - 3s - loss: 0.7639 - accuracy: 0.6772 - 3s/epoch - 15ms/step
744/744 - 119s - loss: 0.8226 - accuracy: 0.6457 - 119s/epoch - 159ms/step
186/186 - 3s - loss: 0.7727 - accuracy: 0.6778 - 3s/epoch - 15ms/step
744/744 - 121s - loss: 0.8284 - accuracy: 0.6483 - 121s/epoch - 162ms/step
186/186 - 4s - loss: 0.7408 - accuracy: 0.6878 - 4s/epoch - 22ms/step
744/744 - 117s - loss: 0.8233 - accuracy: 0.6459 - 117s/epoch - 158ms/step
186/186 - 4s - loss: 0.7796 - accuracy: 0.6814 - 4s/epoch - 24ms/step
Epoch 1/2
744/744 - 122s - loss: 0.8273 - accuracy: 0.6466 - 122s/epoch - 163ms/step
Epoch 2/2
744/744 - 116s - loss: 0.6827 - accuracy: 0.7077 - 116s/epoch - 156ms/step
186/186 - 5s - loss: 0.7390 - accuracy: 0.6676 - 5s/epoch - 26ms/step
Epoch 1/2
744/744 - 116s - loss: 0.